In [ ]:
import numpy as np #Trabalhar com matrizes
import pandas as pd 
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import gensim.downloader as api
from sklearn.manifold import TSNE
import re #Regex
import nltk
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Download de recursos NLTK (se necessário)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
# Dados de exemplo - críticas de filmes (simplificadas)
textos = [
    "Este filme é incrível, adorei a atuação do protagonista",
    "A direção de fotografia é espetacular e o roteiro é envolvente",
    "Péssimo filme, desperdicei meu tempo assistindo isso",
    "Os atores são talentosos mas o roteiro é fraco",
    "Cinematografia belíssima, recomendo assistir no cinema",
    "Não gostei da história, personagens mal desenvolvidos",
    "A trilha sonora combina perfeitamente com as cenas",
    "Filme entediante, previsível do início ao fim",
    "Os efeitos especiais são impressionantes, tecnologia de ponta",
    "História emocionante, chorei no final do filme"
]

# Verificando os dados
for i, texto in enumerate(textos[:3]):  # Mostrando apenas os 3 primeiros
    print(f"Texto {i+1}: {texto}")

Texto 1: Este filme é incrível, adorei a atuação do protagonista
Texto 2: A direção de fotografia é espetacular e o roteiro é envolvente
Texto 3: Péssimo filme, desperdicei meu tempo assistindo isso


In [6]:
from nltk.corpus import stopwords

def preprocessar_texto(texto):
    # Converter para minúsculas
    texto = texto.lower()

    # Remover caracteres especiais e números
    texto = re.sub(r'[^a-záàâãéèêíïóôõöúçñ ]', '', texto)

    # Tokenizar
    tokens = word_tokenize(texto)

    # Remover stopwords (opcional, dependendo da aplicação)
    stop_words = set(stopwords.words('portuguese'))
    tokens = [token for token in tokens if token not in stop_words]

    return tokens

# Aplicar pré-processamento a todos os textos
textos_preprocessados = [preprocessar_texto(texto) for texto in textos]

# Verificar resultado
print("Exemplo de texto original:")
print(textos[0])
print("\nDepois do pré-processamento:")
print(textos_preprocessados[0])

Exemplo de texto original:
Este filme é incrível, adorei a atuação do protagonista

Depois do pré-processamento:
['filme', 'incrível', 'adorei', 'atuação', 'protagonista']


In [7]:
# Definir parâmetros do modelo
vector_size = 100    # Dimensionalidade dos vetores
window = 5           # Tamanho da janela de contexto
min_count = 1        # Frequência mínima das palavras
workers = 4          # Número de threads para treinamento
sg = 1               # Modelo Skip-gram (1) ou CBOW (0

In [8]:
# Treinar o modelo
model = Word2Vec(
    sentences=textos_preprocessados,
    vector_size=vector_size,
    window=window,
    min_count=min_count,
    workers=workers,
    sg=sg
)


In [ ]:
print(f"Modelo treinado com {len(model.wv.key_to_index)} palavras no vocabulário")

Modelo treinado com 44 palavras no vocabulário


In [10]:
# Listar algumas palavras do vocabulário
palavras = list(model.wv.key_to_index.keys())
print("Algumas palavras do vocabulário:")
print(palavras[:10])  # Primeiras 10 palavras

Algumas palavras do vocabulário:
['filme', 'história', 'roteiro', 'desperdicei', 'belíssima', 'cinematografia', 'fraco', 'talentosos', 'atores', 'assistindo']


In [13]:
if 'filme' in model.wv:
    vector_size = palavras
    print(vector_size)

['filme', 'história', 'roteiro', 'desperdicei', 'belíssima', 'cinematografia', 'fraco', 'talentosos', 'atores', 'assistindo', 'tempo', 'péssimo', 'assistir', 'envolvente', 'espetacular', 'fotografia', 'direção', 'protagonista', 'atuação', 'adorei', 'incrível', 'recomendo', 'final', 'chorei', 'previsível', 'emocionante', 'ponta', 'tecnologia', 'impressionantes', 'especiais', 'efeitos', 'fim', 'início', 'entediante', 'gostei', 'cenas', 'perfeitamente', 'combina', 'sonora', 'trilha', 'desenvolvidos', 'mal', 'personagens', 'cinema']


In [14]:
# Encontrar palavras mais similares a 'filme'
if 'filme' in model.wv:
    similares = model.wv.most_similar('filme', topn=5)
    print("\nPalavras mais similares a 'filme':")
    for palavra, similaridade in similares:
        print(f"{palavra}: {similaridade:.4f}")



Palavras mais similares a 'filme':
direção: 0.2189
assistindo: 0.2168
impressionantes: 0.1955
cinema: 0.1694
efeitos: 0.1520


In [16]:
model.save('vasco.model')

In [19]:

# Listar modelos disponíveis
print("Alguns modelos disponíveis no gensim-data:")
print([nome for nome in list(api.info()['models'].keys())[:10]])

# Carregar um modelo pré-treinado (word2vec-google-news-300)
# Nota: Isso pode demorar um pouco na primeira execução (download)
try:
    # Para português, você pode tentar 'word2vec-portuguese' se disponível
    # ou usar modelos do NILC: http://nilc.icmc.usp.br/embeddings
    modelo_pretreino = api.load("glove-wiki-gigaword-100")  # Mais rápido que word2vec-google-news-300
    print(f"Modelo pré-treinado carregado com {len(modelo_pretreino.key_to_index)} palavras")

    # Verificar palavras similares usando modelo pré-treinado
    if 'computer' in modelo_pretreino:
        similares = modelo_pretreino.most_similar('computer', topn=5)
        print("\nPalavras mais similares a 'computer':")
        for palavra, similaridade in similares:
            print(f"{palavra}: {similaridade:.4f}")
except Exception as e:
    print(f"Erro ao carregar modelo pré-treinado: {e}")
    print("Tente outro modelo ou prossiga sem esta parte")


Alguns modelos disponíveis no gensim-data:
['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50']
[==================================================] 100.0% 128.1/128.1MB downloaded
Modelo pré-treinado carregado com 400000 palavras

Palavras mais similares a 'computer':
computers: 0.8752
software: 0.8373
technology: 0.7642
pc: 0.7366
hardware: 0.7290


In [20]:
# Usando modelo pré-treinado para analogias (se disponível)
try:
    # Famosa analogia: rei - homem + mulher = rainha
    if all(word in modelo_pretreino for word in ['king', 'man', 'woman']):
        resultado = modelo_pretreino.most_similar(
            positive=['king', 'woman'],
            negative=['man'],
            topn=3
        )
        print("\nAnalogia: rei - homem + mulher =")
        for palavra, similaridade in resultado:
            print(f"{palavra}: {similaridade:.4f}")
except:
    print("Não foi possível realizar a operação de analogia com o modelo disponível")

# Podemos tentar outras analogias com nosso modelo treinado
# Por exemplo: bom - positivo + negativo = ruim
# (Dependendo do tamanho do corpus, pode não funcionar bem para modelos pequenos)



Analogia: rei - homem + mulher =
queen: 0.7699
monarch: 0.6843
throne: 0.6756


In [21]:

# Listar modelos disponíveis
print("Alguns modelos disponíveis no gensim-data:")
print([nome for nome in list(api.info()['models'].keys())[:10]])

# Carregar um modelo pré-treinado (word2vec-google-news-300)
# Nota: Isso pode demorar um pouco na primeira execução (download)
try:
    # Para português, você pode tentar 'word2vec-portuguese' se disponível
    # ou usar modelos do NILC: http://nilc.icmc.usp.br/embeddings
    modelo_pretreino = api.load("glove-wiki-gigaword-100")  # Mais rápido que word2vec-google-news-300
    print(f"Modelo pré-treinado carregado com {len(modelo_pretreino.key_to_index)} palavras")

    # Verificar palavras similares usando modelo pré-treinado
    if 'computer' in modelo_pretreino:
        similares = modelo_pretreino.most_similar('computer', topn=5)
        print("\nPalavras mais similares a 'computer':")
        for palavra, similaridade in similares:
            print(f"{palavra}: {similaridade:.4f}")
except Exception as e:
    print(f"Erro ao carregar modelo pré-treinado: {e}")
    print("Tente outro modelo ou prossiga sem esta parte")


Alguns modelos disponíveis no gensim-data:
['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50']
Modelo pré-treinado carregado com 400000 palavras

Palavras mais similares a 'computer':
computers: 0.8752
software: 0.8373
technology: 0.7642
pc: 0.7366
hardware: 0.7290


In [22]:
# Função para calcular similaridade entre pares de palavras
def calcular_similaridade(modelo, pares_palavras):
    resultados = []
    for par in pares_palavras:
        palavra1, palavra2 = par
        if palavra1 in modelo.wv and palavra2 in modelo.wv:
            similaridade = modelo.wv.similarity(palavra1, palavra2)
            resultados.append((par, similaridade))
        else:
            resultados.append((par, "Uma ou ambas as palavras não estão no vocabulário"))
    return resultados

# Pares de palavras para testar
pares = [
    ('filme', 'cinema'),
    ('bom', 'ruim'),
    ('ator', 'atuação'),
    ('filme', 'protagonista')
]


In [23]:
# Calcular similaridades
similaridades = calcular_similaridade(model, pares)

# Exibir resultados
print("\nSimilaridade entre pares de palavras:")
for (palavra1, palavra2), similaridade in similaridades:
    if isinstance(similaridade, float):
        print(f"{palavra1} - {palavra2}: {similaridade:.4f}")
    else:
        print(f"{palavra1} - {palavra2}: {similaridade}")



Similaridade entre pares de palavras:
filme - cinema: 0.1693534404039383
bom - ruim: Uma ou ambas as palavras não estão no vocabulário
ator - atuação: Uma ou ambas as palavras não estão no vocabulário
filme - protagonista: -0.11359719187021255


In [24]:
def visualizar_embeddings(modelo, palavras=None, n_palavras=50):
    """
    Visualiza os embeddings em um espaço 2D usando t-SNE.
    Args:
        modelo: Modelo Word2Vec
        palavras: Lista de palavras específicas para visualizar (opcional)
        n_palavras: Número de palavras mais frequentes a visualizar (se palavras=None)
    """
    # Obter palavras e vetores
    if palavras is None:
        palavras = list(modelo.wv.key_to_index.keys())[:n_palavras]
    else:
        # Filtrar palavras que não estão no vocabulário
        palavras = [p for p in palavras if p in modelo.wv]
    
    if not palavras:
        print("Nenhuma palavra válida para visualização")
        return
    
    # Obter vetores e converter para array NumPy
    vetores = np.array([modelo.wv[palavra] for palavra in palavras])
    
    # Reduzir dimensionalidade com t-SNE
    tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(palavras)-1))
    vetores_2d = tsne.fit_transform(vetores)
    
    # Criar gráfico
    plt.figure(figsize=(12, 10))
    
    # Plotar pontos
    x = vetores_2d[:, 0]
    y = vetores_2d[:, 1]
    plt.scatter(x, y, c='blue', alpha=0.7)
    
    # Adicionar rótulos (palavras)
    for i, palavra in enumerate(palavras):
        plt.annotate(
            palavra,
            xy=(x[i], y[i]),
            xytext=(5, 2),
            textcoords='offset points',
            fontsize=10
        )
    
    plt.title("Visualização t-SNE dos Word Embeddings")
    plt.grid(True)
    plt.show()

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

# Dados rotulados para exemplo
textos_rotulados = textos  # Usando os mesmos textos de antes
sentimentos = [1, 1, 0, 0, 1, 0, 1, 0, 1, 1]  # 1: positivo, 0: negativo

# Função para gerar vetores de documento usando embeddings
def texto_para_vetor(texto, modelo):
    """Converte um texto em um vetor médio dos embeddings de suas palavras"""
    palavras = preprocessar_texto(texto)
    # Filtrar palavras que estão no vocabulário do modelo
    palavras_no_vocab = [p for p in palavras if p in modelo.wv]
    if not palavras_no_vocab:
        # Se nenhuma palavra estiver no vocabulário, retorna vetor de zeros
        return np.zeros(modelo.vector_size)
    # Calcular a média dos vetores das palavras
    vetores = [modelo.wv[palavra] for palavra in palavras_no_vocab]
    return np.mean(vetores, axis=0)

# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    textos_rotulados, sentimentos, test_size=0.3, random_state=42
)

# 1. Abordagem com TF-IDF
vectorizer = TfidfVectorizer(max_features=100)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

clf_tfidf = LogisticRegression(random_state=42)
clf_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = clf_tfidf.predict(X_test_tfidf)

# 2. Abordagem com Word Embeddings
X_train_emb = np.array([texto_para_vetor(texto, model) for texto in X_train])
X_test_emb = np.array([texto_para_vetor(texto, model) for texto in X_test])

clf_emb = LogisticRegression(random_state=42)
clf_emb.fit(X_train_emb, y_train)
y_pred_emb = clf_emb.predict(X_test_emb)

# Comparar resultados
print("\nResultados com TF-IDF:")
print(classification_report(y_test, y_pred_tfidf))

print("\nResultados com Word Embeddings:")
print(classification_report(y_test, y_pred_emb))



Resultados com TF-IDF:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.67      1.00      0.80         2

    accuracy                           0.67         3
   macro avg       0.33      0.50      0.40         3
weighted avg       0.44      0.67      0.53         3


Resultados com Word Embeddings:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.67      1.00      0.80         2

    accuracy                           0.67         3
   macro avg       0.33      0.50      0.40         3
weighted avg       0.44      0.67      0.53         3

